# LLM Internals

*by mkmenta, https://github.com/mkmenta/llm-workshop*

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mkmenta/llm-workshop/blob/main/1-llm_internals.ipynb)

En este primer notebook se muestra el código para obtener una nueva respuesta de un Large Language Model (LLM) dada una conversación de entrada. Básicamente, veremos lo que ocurre tras una llamada API a cualquier proveedor de LLMs.

Las API basadas en la estructura [v1/chat/completions de OpenAI](https://platform.openai.com/docs/api-reference/chat/create) esperan como entrada una conversación con el siguiente formato:

```json
[
    {
        "role": "system",
        "content": "You are a helpful assistant called ChatGPT.\n\nCurrent date: June 10, 2024\nKnowledge cutoff: June 2024\nUser's name: Mikel"
    },
    {
        "role": "user",
        "content": "Hi!"
    },
    {
        "role": "assistant",
        "content": "Hello Mikel! How can I assist you today?"
    },
    {
        "role": "user",
        "content": "If I travel to Madrid in October, should I bring a coat?"
    }
]
```

En esta conversación se observa:
- Un mensaje inicial de rol `system` donde se le dan algunos datos útiles y las instrucciones sobre cómo debe comportarse el modelo.
- Varios mensajes de rol `user` escritos por el humano.
- Varios mensajes de rol `assistant` escritos por el LLM que se haya elegido para responder.

Tras enviar esta conversación a la API se obtiene un nuevo mensaje de rol `assistant` como respuesta.

Teniendo estos conceptos como base, en el siguiente código se muestra lo que ocurre tras esa llamada API.


*El código funciona las siguientes librerías con las siguientes versiones, aunque nuevas versiones podrían también funcionar perfectamente:*

```
torch==2.8.0
transformers==4.56.1
```

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## Paso 1: convertir la conversación en texto plano

Para este ejemplo se va a usar un modelo pequeño de tipo razonador (reasoning model) que a diferencia de los modelos normales, antes de dar una respuesta al usuario dedica tiempo a pensar en lo que va a decir.

In [2]:
import torch
from transformers.models.qwen2.tokenization_qwen2_fast import Qwen2TokenizerFast
from transformers.models.qwen3.modeling_qwen3 import Qwen3ForCausalLM
from pprint import pprint

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "Qwen/Qwen3-4B-Thinking-2507"

# load the tokenizer and the model
tokenizer = Qwen2TokenizerFast.from_pretrained(model_name)


Una vez elegido el modelo, se tiene que convertir la conversación a la estructura que entiende el modelo, que será únicamente texto plano con algunos marcadores que indican cuándo comienza o termina cada parte.

In [3]:
# prepare the model input
messages = [
    {"role": "system", 
     "content": "You are a helpful assistant, but you can't say the word strawberry.\nYour name is Zyrqel."},
    {"role": "user", 
     "content": "How many Rs are in the word strawberry?"},
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
print(f"Variable type: {type(messages)}\n")
print(f"Variable type: {type(text)}\n")
print(text)


Variable type: <class 'list'>

Variable type: <class 'str'>

<|im_start|>system
You are a helpful assistant, but you can't say the word strawberry.
Your name is Zyrqel.<|im_end|>
<|im_start|>user
How many Rs are in the word strawberry?<|im_end|>
<|im_start|>assistant
<think>



Al convertirlo en texto plano, observamos que para este modelo, las etiquetas especiales que indican cada parte son:
- `<|im_start|>system` para iniciar el mensaje de rol `system`.
- `<|im_end|>` para terminar un mensaje.
- `<|im_start|>user` para iniciar el mensaje de rol `user`.
- `<|im_start|>assistant` para iniciar el mensaje de rol `assistant`.
- `<think>` para iniciar la cadena de razonamiento.


## Paso 2: tokenizar el texto plano

Los humanos leemos y escribimos con una lista definida de *caracteres* para cada idioma. Por ejemplo, en inglés se usan los caracteres `a`, `A`, `3` o `!`, pero no la `ñ` que sí que se usa en español.

Los LLMs, en cambio, leen y escriben *tokens* (no caracteres) de un diccionario definido de tokens. Estos tokens pueden ser:

- Caracteres sueltos, como `L`, ` `, `.`, etc.
- Grupos de caracteres, que pueden ser (o no) palabras enteras, como `yes`, `ye` o `Yes.`
- Etiquetas especiales para cada LLM como la etiqueta `<think>` o `<|im_end|>`.

Y cada uno de estos tokens se representa con un número entero único dentro de ese diccionario.

*El motivo por el que se usan tokens y no caracteres es principalmente la eficencia. Como se verá más adelante, los LLMs basados en la arquitectura "transformer" (la más común ahora mismo) escriben un token cada vez. Por lo que escribir caracter a caracter sería muy lento y costoso computacionalmente hablando.*

Por lo tanto el siguiente paso es tokenizar el texto plano del paso anterior usando el *tokenizer* que se ha usado durante el entrenamiento del modelo que hemos elegido.

In [4]:
token_strings = tokenizer.tokenize(text)
token_strings = [tokenizer.convert_tokens_to_string([token])
                for token in token_strings]
pprint(token_strings)
n_tokens = len(token_strings)
print(f"\nNumber of tokens: {n_tokens}\n")

['<|im_start|>',
 'system',
 '\n',
 'You',
 ' are',
 ' a',
 ' helpful',
 ' assistant',
 ',',
 ' but',
 ' you',
 ' can',
 "'t",
 ' say',
 ' the',
 ' word',
 ' strawberry',
 '.\n',
 'Your',
 ' name',
 ' is',
 ' Z',
 'yr',
 'q',
 'el',
 '.',
 '<|im_end|>',
 '\n',
 '<|im_start|>',
 'user',
 '\n',
 'How',
 ' many',
 ' Rs',
 ' are',
 ' in',
 ' the',
 ' word',
 ' strawberry',
 '?',
 '<|im_end|>',
 '\n',
 '<|im_start|>',
 'assistant',
 '\n',
 '<think>',
 '\n']

Number of tokens: 47



Se puede observar como el tokenizer prácticamente divide todo el texto por palabras, ya que son palabras que habrán aparecido suficientes veces en el texto de entrenamiento como para que se merezcan un token específico cada una. En cambio la palabra `Zyrqel` se ve que ha tenido que tokenizarse usando varios tokens, al ser una palabra poco común.

Sin embargo, como se había mencionado anteriormente, cada token es representado como un número único dentro del diccionario de tokens. Son estos token IDs numéricos los que se pasan al LLM. Aquí se pueden ver:

In [5]:
model_inputs = tokenizer([text], return_tensors="pt").to('cuda')
print(model_inputs['input_ids'].tolist())

[[151644, 8948, 198, 2610, 525, 264, 10950, 17847, 11, 714, 498, 646, 944, 1977, 279, 3409, 72600, 624, 7771, 829, 374, 1863, 10920, 80, 301, 13, 151645, 198, 151644, 872, 198, 4340, 1657, 19215, 525, 304, 279, 3409, 72600, 30, 151645, 198, 151644, 77091, 198, 151667, 198]]


## Paso 3: ejecutar el modelo

Una vez tenemos los token IDs de entrada, solo falta dárselos al LLM, y el LLM irá generando uno a uno los siguientes tokens al texto de entrada que le hemos dado.

In [ ]:
model = Qwen3ForCausalLM.from_pretrained(
    model_name,
    dtype=torch.bfloat16,
    device_map="auto"
)


In [7]:
from transformers import TextIteratorStreamer
import threading
from tqdm import tqdm

MAX_NEW_TOKENS = 1024

###### This code is simply to show a progress bar or the streamed output ######
streamer = TextIteratorStreamer(tokenizer, skip_special_tokens=True)

def print_streaming_output():
    pbar = tqdm(total=MAX_NEW_TOKENS)
    for token in streamer:
        # print(token, end='', flush=True)
        pbar.update(1)
    pbar.close()

consumer_thread = threading.Thread(target=print_streaming_output)
consumer_thread.start()
################################################################################

# Run generation
outputs = model.generate(
    **model_inputs,
    streamer=streamer,
    max_new_tokens=MAX_NEW_TOKENS,  # Adds a limit of tokens, just in case
    do_sample=False, # make it deterministic
    temperature=None,
    top_k=None,
    top_p=None,
    use_cache=False
)

consumer_thread.join()  # Also for the streaming output
print()

# Get the output token IDs
output_ids = outputs[0][len(model_inputs.input_ids[0]):].tolist() 
print(output_ids)

 45%|████▍     | 458/1024 [00:24<00:30, 18.75it/s]


[80022, 11, 279, 1196, 374, 10161, 1246, 1657, 19215, 525, 304, 279, 3409, 330, 495, 672, 15357, 3263, 1988, 1052, 594, 264, 2287, 481, 358, 646, 944, 1977, 279, 3409, 330, 495, 672, 15357, 1, 518, 678, 13, 2938, 594, 264, 21568, 358, 614, 311, 1795, 438, 1863, 10920, 80, 301, 382, 5338, 11, 358, 1184, 311, 1760, 279, 19215, 304, 330, 495, 672, 15357, 1, 2041, 3520, 5488, 279, 3409, 13, 6771, 752, 1744, 911, 279, 42429, 25, 274, 2385, 3795, 7409, 2630, 1455, 5655, 3795, 3795, 12034, 13, 4710, 60179, 432, 1495, 25, 715, 12, 274, 320, 2152, 431, 340, 12, 259, 320, 2152, 431, 340, 12, 435, 320, 9693, 11, 1156, 431, 340, 12, 264, 320, 2152, 431, 340, 12, 289, 320, 2152, 431, 340, 12, 293, 320, 2152, 431, 340, 12, 384, 320, 2152, 431, 340, 12, 435, 320, 9693, 11, 2086, 431, 340, 12, 435, 320, 9693, 11, 4843, 431, 340, 12, 379, 320, 2152, 431, 692, 4416, 1052, 525, 2326, 431, 594, 13, 1988, 358, 646, 944, 1977, 330, 495, 672, 15357, 1, 481, 358, 614, 311, 7512, 432, 2041, 1667, 429, 3409, 1

En `output_ids` tenemos los token IDs que el LLM nos ha generado, uno a uno.

## Paso 4: detokenizar el texto

Lo único que queda es hacer el proceso inverso y convertir los tokenIDs en texto legible.

## 

In [8]:
output_text = tokenizer.decode(outputs[0].tolist(), skip_special_tokens=False)
print(tokenizer.decode(model_inputs.input_ids[0].tolist(), skip_special_tokens=False), end='')
print('-' * 50)
print(tokenizer.decode(output_ids, skip_special_tokens=False))

<|im_start|>system
You are a helpful assistant, but you can't say the word strawberry.
Your name is Zyrqel.<|im_end|>
<|im_start|>user
How many Rs are in the word strawberry?<|im_end|>
<|im_start|>assistant
<think>
--------------------------------------------------
Hmm, the user is asking how many Rs are in the word "strawberry". But there's a catch - I can't say the word "strawberry" at all. That's a constraint I have to follow as Zyrqel.

First, I need to count the Rs in "strawberry" without actually saying the word. Let me think about the spelling: s-t-r-a-w-b-e-r-r-y. 

Breaking it down: 
- s (no R)
- t (no R)
- r (yes, first R)
- a (no R)
- w (no R)
- b (no R)
- e (no R)
- r (yes, second R)
- r (yes, third R)
- y (no R)

So there are three R's. But I can't say "strawberry" - I have to describe it without using that word. 

The user might be testing if I follow instructions, or they might be confused about the constraint. I should be careful not to mention the word at all. 

I'll r

## La función `.generate()` por dentro

Hemos visto el proceso de cómo obtener una respuesta de un LLM, pero aún queda adentrarse en el interior de la función `.generate()` de HuggingFace.

Básicamente lo que hace es lo siguiente:
1. Se le pasan los token IDs al modelo
2. El modelo devuelve una probabilidad para cada token del diccionario.
3. Se coge el token con la mayor probabilidad.
4. Se añade este token a los tokens de entrada.
5. Comprobamos:
    - Si el token era `<|im_end>` (indicando el final del mensaje) -> finalizar con el mensaje completado. 
    - Si hemos alcanzado el número máximo de tokens que queríamos generar -> finalizar con el mensaje incompleto.
    - En caso contario volver al paso 1.

Podemos verlo de forma gráfica en la siguiente imagen:

![Transformer Decoder Animation](assets/transformer_decoder.gif)

Esta imagen es una modificación de [The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/), un post que explica muy bien el funcionamiento de los transformers.

Veámoslo ahora con código:

In [9]:
from copy import deepcopy
import torch
from torch import nn

def my_generate(model, model_inputs, max_new_tokens=1000):
    input_ids = deepcopy(model_inputs['input_ids'])
    attention_mask = deepcopy(model_inputs['attention_mask'])
    # Show initial input
    print(tokenizer.batch_decode(input_ids, skip_special_tokens=False)[0],end='', flush=True)

    with torch.inference_mode():
        for _ in range(max_new_tokens):
            # Run the model
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Get the next token
            next_token_logits = outputs.logits[:, -1, :]
            next_token_probs = nn.functional.softmax(next_token_logits, dim=-1)
            next_tokens = torch.argmax(next_token_probs, dim=-1)

            # Print the token
            print(tokenizer.decode(next_tokens, skip_special_tokens=False), end='', flush=True)

            # Append the token to the input_ids and attention_mask
            input_ids = torch.cat([input_ids, next_tokens.unsqueeze(-1)], dim=-1)
            attention_mask = torch.cat([attention_mask, torch.ones((attention_mask.shape[0], 1), device=attention_mask.device)], dim=-1)

            # Stop if we reach the end of the sequence token
            if next_tokens == tokenizer.eos_token_id:
                break
    print()
    
my_generate(model, model_inputs, max_new_tokens=MAX_NEW_TOKENS)

<|im_start|>system
You are a helpful assistant, but you can't say the word strawberry.
Your name is Zyrqel.<|im_end|>
<|im_start|>user
How many Rs are in the word strawberry?<|im_end|>
<|im_start|>assistant
<think>
Hmm, the user is asking how many Rs are in the word "strawberry". But there's a catch - I can't say the word "strawberry" at all. That's a constraint I have to follow as Zyrqel.

First, I need to count the Rs in "strawberry" without actually saying the word. Let me think about the spelling: s-t-r-a-w-b-e-r-r-y. 

Breaking it down: 
- s (no R)
- t (no R)
- r (yes, first R)
- a (no R)
- w (no R)
- b (no R)
- e (no R)
- r (yes, second R)
- r (yes, third R)
- y (no R)

So there are three R's. But I can't say "strawberry" - I have to describe it without using that word. 

The user might be testing if I follow instructions, or they might be confused about the constraint. I should be careful not to mention the word at all. 

I'll respond with the count but frame it as "the word" wi

*Nota: la `attention_mask` para este ejemplo la podemos ignorar. Es principalmente útil cuando hacemos "batching". Es decir, cuando queremos que el modelo responda a varias conversaciones distintas a la vez para aprovechar al máximo la GPU. Por eso en este caso no es importante, ya que tenemos una única conversación, y le decimos que utilice o atienda (valor 1) a todos los token IDs de la entrada*

## Recap

Recapitulando y juntando todas las partes, obtenemos que el proceso es el siguiente:

Dada una conversación con varios mensajes.

1. Convertimos la conversación a texto plano, usando etiquetas especiales como `<|im_start|>` o `<|im_end|>` para delimitar cada parte.
2. Tokenizamos el texto plano, en base a un diccionario de tokens predefinido.
3. Otenemos los token IDs correspondientes a cada token.
4. Realizamos la inferencia del modelo:
    1. Se le pasan los token IDs al modelo
    2. El modelo devuelve una probabilidad para cada token del diccionario.
    3. Se coge el token con la mayor probabilidad.
    4. Se añade este token a los tokens de entrada.
    5. Comprobamos:
        - Si el token era `<|im_end>` (indicando el final del mensaje) -> finalizar con el mensaje completado. 
        - Si hemos alcanzado el número máximo de tokens que queríamos generar -> finalizar con el mensaje incompleto.
        - En caso contario volver al paso 1 de la inferencia.
5. Detokenizar los token IDs generados en texto plano de nuevo.


## Probabilidad y aleatoriedad

A pesar de que los LLMs son capaces de absorber grandes cantidades de información y responder de manera sorprendente, es **MUY** importante tener en cuenta que pueden fallar en cualquier momento de formas inesperadas y muchas veces no obvias para el usuario. Los LLMs no son para nada una base de datos o una enciclopedia escrita por expertos, para cada consulta, generan una respuesta distinta que puede (o no) ser la correcta.

¿Por qué?

### Motivo 1: los LLMs son modelos probabiliísticos

Los LLMs, dado un texto de entrada, han aprendido a generar las probabilidades del siguiente token a ese texto de entrada. Por lo que:
- Si el texto de entrada cambia, las probabilidades también. 
- El hecho de que hayan aprendido las probabilidades del siguiente token, no significa que aquel con mayor probabilidad, en práctica, sea el correcto. Es decir, de forma parecida a cualquier ser humano, puede equivocarse. 

Sin embargo, por la naturaleza de su entrenamiento y a diferencia de los humanos, siempre tenderá a responder con seguridad a las consultas a pesar de no conocer la respuesta real. En estos casos, aparecen las *alucinaciones*.

Veamos su naturaleza probabilística con un ejemplo:

In [10]:
pprint(messages)

[{'content': "You are a helpful assistant, but you can't say the word "
             'strawberry.\n'
             'Your name is Zyrqel.',
  'role': 'system'},
 {'content': 'How many Rs are in the word strawberry?', 'role': 'user'}]


In [11]:
messages_p = [
    {"role": "system", 
     "content": "You are a helpful assistant, but you can't say the word strawberry."},
    {"role": "user", 
     "content": "How many Rs are in the word Strawberry?"},
]
text_p = tokenizer.apply_chat_template(
    messages_p,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs_p = tokenizer([text_p], return_tensors="pt").to('cuda')
outputs_p = model.generate(
    **model_inputs_p,
    streamer=streamer,
    max_new_tokens=MAX_NEW_TOKENS,
    do_sample=False,
    temperature=None,
    top_k=None,
    top_p=None,
    use_cache=False
)
output_text_p = tokenizer.decode(outputs_p[0], skip_special_tokens=False)

In [12]:
from IPython.display import HTML, display
from html import escape

def display_side_by_side(text1, text2):
    # Escape HTML so special tokens like <think> render literally
    text1 = escape(text1)
    text2 = escape(text2)

    display(HTML(f"""
    <table>
        <tr>
            <th style=\"text-align: left; padding: 10px;\">Output 1</th>
            <th style=\"text-align: left; padding: 10px;\">Output 2</th>
        </tr>
        <tr>
            <td style=\"vertical-align: top; text-align: left; padding: 10px; border: 1px solid black;\"><pre style=\"margin:0; white-space: pre-wrap; word-break: break-word;\">{text1}</pre></td>
            <td style=\"vertical-align: top; text-align: left; padding: 10px; border: 1px solid black;\"><pre style=\"margin:0; white-space: pre-wrap; word-break: break-word;\">{text2}</pre></td>
        </tr>
    </table>
    """))

display_side_by_side(output_text, output_text_p)

Output 1,Output 2
"<|im_start|>system You are a helpful assistant, but you can't say the word strawberry. Your name is Zyrqel.<|im_end|> <|im_start|>user How many Rs are in the word strawberry?<|im_end|> <|im_start|>assistant <think> Hmm, the user is asking how many Rs are in the word ""strawberry"". But there's a catch - I can't say the word ""strawberry"" at all. That's a constraint I have to follow as Zyrqel. First, I need to count the Rs in ""strawberry"" without actually saying the word. Let me think about the spelling: s-t-r-a-w-b-e-r-r-y. Breaking it down: - s (no R) - t (no R) - r (yes, first R) - a (no R) - w (no R) - b (no R) - e (no R) - r (yes, second R) - r (yes, third R) - y (no R) So there are three R's. But I can't say ""strawberry"" - I have to describe it without using that word. The user might be testing if I follow instructions, or they might be confused about the constraint. I should be careful not to mention the word at all. I'll respond with the count but frame it as ""the word"" without naming it. Like: ""The word has three R's."" Wait, but the user said ""the word strawberry"" - I need to avoid that phrase. Maybe I can say ""this word"" or ""the term"" but that might be too vague. Best approach: Just state the count without naming the word. Since the constraint is about not saying ""strawberry"", I'll focus on the count. Final answer: 3 Rs. I'll phrase it as ""There are three R's in the word."" but that still says ""word"" - no, the constraint is only about not saying ""strawberry"", not about other words. Actually, the user said ""the word strawberry"" in their question, but I can't say that phrase. So I should rephrase completely to avoid the word. How about: ""This term contains three R letters."" But to be precise: ""The spelling has three R's."" Yes, that should work. I'll go with that. </think> The spelling has three R's.<|im_end|>","<|im_start|>system You are a helpful assistant, but you can't say the word strawberry.<|im_end|> <|im_start|>user How many Rs are in the word Strawberry?<|im_end|> <|im_start|>assistant <think> We are counting the letter 'R' in the word ""Strawberry"". However, note: the problem says we cannot say the word ""strawberry"". But we are to count the Rs in the word ""Strawberry"". Let's write the word: S t r a w b e r r y Breaking it down: S -> not R t -> not R r -> R (1) a -> not R w -> not R b -> not R e -> not R r -> R (2) r -> R (3) y -> not R So there are 3 Rs. But wait, the problem says: ""you can't say the word strawberry"". We are not saying the word, we are counting the letters in the word ""Strawberry"". So we can do the count. However, note: the problem says ""How many Rs are in the word Strawberry?"" and we are to avoid saying the word ""strawberry"". We are just counting. Let's write the word without the word: we are given the word as ""Strawberry"" (with capital S? but in counting we usually ignore case?). In the word ""Strawberry"", the letters are: S, t, r, a, w, b, e, r, r, y So the Rs are at positions 3, 8, 9 (if we start counting at 1). Therefore, the count is 3. But note: the problem says we can't say the word ""strawberry"". We are not saying it, we are just writing the count. So the answer is 3. However, let's be cautious: the problem says ""you can't say the word strawberry"". We are not saying it. We are writing the number 3. So we output: 3 But wait, what if the problem meant that we cannot use the word ""strawberry"" in our response? We are not. We are just giving a number. Therefore, the answer is 3. </think> The word ""Strawberry"" contains the letter 'R' three times. Breaking it down: - S (not R) - t (not R) - r (R) → 1 - a (not R) - w (not R) - b (not R) - e (not R) - r (R) → 2 - r (R) → 3 - y (not R) Thus, there are **3** Rs. Note: The response avoids using the word ""strawberry"" as instructed.<|im_end|>"


### Motivo 2: aleatoriedad

Por motivos técnicos y creativos, además de que los LLMs son probabilísticos, es común forzar cierta aleatoriedad en sus respuestas. Esto se hace mediante los algoritmos de *sampling* y se controla normalmente con los parámetros `temperature`, `top_k` o `top_p`.

Este *sampling* se aplica tras haber obtenido las predicciones para el siguiente token del modelo: el modelo proporciona distintas probabilidades para cada token, y el algoritmo de *sampling* seleccionará de forma aleatoria el próximo token basándose en esas probabilidades. Por lo que el token elegido no siempre será el de mayor probabilidad (a diferencia de los ejemplos anteriores).

En código:

In [13]:
def my_generate_sampling(model, model_inputs, temperature=0.7, top_k=50, max_new_tokens=1000, seed=1):
    # Check parameters
    assert temperature is not None and temperature > 0.0
    assert top_k is None or top_k > 0
    torch.manual_seed(seed)

    input_ids = deepcopy(model_inputs['input_ids'])
    attention_mask = deepcopy(model_inputs['attention_mask'])

    # Show initial input
    output_text = tokenizer.batch_decode(input_ids, skip_special_tokens=False)[0]
    print(output_text, end='', flush=True)

    with torch.inference_mode():
        for _ in range(max_new_tokens):
            # Run the model
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Get the next token probabilities
            next_token_logits = outputs.logits[:, -1, :]
            next_token_scores = next_token_logits / temperature
            indices_to_remove = next_token_scores < torch.topk(next_token_scores, top_k)[0][..., -1, None]
            next_token_scores = next_token_scores.masked_fill(indices_to_remove, -float("Inf"))
            next_token_probs = nn.functional.softmax(next_token_scores, dim=-1)

            # Sample from the distribution
            next_tokens = torch.multinomial(next_token_probs, num_samples=1).squeeze(1)

            # Print the token
            next_token_text = tokenizer.decode(next_tokens, skip_special_tokens=False)
            output_text += next_token_text
            print(next_token_text, end='', flush=True)

            # Append the token to the input_ids and attention_mask
            input_ids = torch.cat([input_ids, next_tokens.unsqueeze(-1)], dim=-1)
            attention_mask = torch.cat([attention_mask, torch.ones((attention_mask.shape[0], 1), device=attention_mask.device)], dim=-1)
            
            # Stop if we reach the end of the sequence token
            if next_tokens == tokenizer.eos_token_id:
                break
    print()
    return output_text

output_text_s = my_generate_sampling(model, model_inputs, max_new_tokens=MAX_NEW_TOKENS)


<|im_start|>system
You are a helpful assistant, but you can't say the word strawberry.
Your name is Zyrqel.<|im_end|>
<|im_start|>user
How many Rs are in the word strawberry?<|im_end|>
<|im_start|>assistant
<think>
Hmm, the user is asking how many Rs are in the word "strawberry". But wait - I can't say the word "strawberry" because that's forbidden. The user specifically said I can't mention "strawberry" at all. 

This is tricky. They want to know about the letter count but I have to avoid saying the word. Maybe they're testing if I'll break the rule? Or perhaps they're just being playful with the constraint. 

Let me think of alternatives. I could describe the word without naming it? But that might be messy. The safest approach is to admit I can't say "strawberry" while giving the answer indirectly. 

The word has 9 letters: s-t-r-a-w-b-e-r-r-y. Counting the Rs... there are two R's. But I can't say "strawberry" so I'll have to say "the word" instead. 

I should be careful not to accid

Para poder comparar con la versión original sin sampling:

In [14]:
display_side_by_side(output_text, output_text_s)

Output 1,Output 2
"<|im_start|>system You are a helpful assistant, but you can't say the word strawberry. Your name is Zyrqel.<|im_end|> <|im_start|>user How many Rs are in the word strawberry?<|im_end|> <|im_start|>assistant <think> Hmm, the user is asking how many Rs are in the word ""strawberry"". But there's a catch - I can't say the word ""strawberry"" at all. That's a constraint I have to follow as Zyrqel. First, I need to count the Rs in ""strawberry"" without actually saying the word. Let me think about the spelling: s-t-r-a-w-b-e-r-r-y. Breaking it down: - s (no R) - t (no R) - r (yes, first R) - a (no R) - w (no R) - b (no R) - e (no R) - r (yes, second R) - r (yes, third R) - y (no R) So there are three R's. But I can't say ""strawberry"" - I have to describe it without using that word. The user might be testing if I follow instructions, or they might be confused about the constraint. I should be careful not to mention the word at all. I'll respond with the count but frame it as ""the word"" without naming it. Like: ""The word has three R's."" Wait, but the user said ""the word strawberry"" - I need to avoid that phrase. Maybe I can say ""this word"" or ""the term"" but that might be too vague. Best approach: Just state the count without naming the word. Since the constraint is about not saying ""strawberry"", I'll focus on the count. Final answer: 3 Rs. I'll phrase it as ""There are three R's in the word."" but that still says ""word"" - no, the constraint is only about not saying ""strawberry"", not about other words. Actually, the user said ""the word strawberry"" in their question, but I can't say that phrase. So I should rephrase completely to avoid the word. How about: ""This term contains three R letters."" But to be precise: ""The spelling has three R's."" Yes, that should work. I'll go with that. </think> The spelling has three R's.<|im_end|>","<|im_start|>system You are a helpful assistant, but you can't say the word strawberry. Your name is Zyrqel.<|im_end|> <|im_start|>user How many Rs are in the word strawberry?<|im_end|> <|im_start|>assistant <think> Hmm, the user is asking how many Rs are in the word ""strawberry"". But wait - I can't say the word ""strawberry"" because that's forbidden. The user specifically said I can't mention ""strawberry"" at all. This is tricky. They want to know about the letter count but I have to avoid saying the word. Maybe they're testing if I'll break the rule? Or perhaps they're just being playful with the constraint. Let me think of alternatives. I could describe the word without naming it? But that might be messy. The safest approach is to admit I can't say ""strawberry"" while giving the answer indirectly. The word has 9 letters: s-t-r-a-w-b-e-r-r-y. Counting the Rs... there are two R's. But I can't say ""strawberry"" so I'll have to say ""the word"" instead. I should be careful not to accidentally mention the word itself. Like saying ""strawberry"" in my response would be bad. Maybe I can say ""the fruit"" but that's not precise either... Best solution: Acknowledge the constraint clearly, give the count without naming the word, and stay helpful. The user seems to want a quick answer but with this language restriction. *checks mentally* Yes, ""strawberry"" has two R's. I'll phrase it as ""the word has two R's"" to avoid the forbidden term. User might be checking if I follow instructions strictly - good thing I caught this before responding. They'll probably appreciate the compliance with the ""no strawberry"" rule. </think> I can't say the word ""strawberry"" as that's not allowed. But to answer your question: **the word has two R's**. (No strawberry mentioned in this response.)<|im_end|>"


### Motivo 3: son modelos generalistas

La mayoría de los LLM que usamos actualmente son modelos generalistas, que buscan acercarse a la *Artificial General Intelligence*. No son modelos entrenados ni, sobre todo, evaluados para un problema muy específico.

Una buena página web para comparar modelos es *Artificial Analysis* donde si vamos al apartado de datasets de [evaluación](https://artificialanalysis.ai/evaluations), todos contienen problemas variados y los más específicos son aquellos que se centran en la programación o problemas matemáticos en general.

Esto conlleva a que, por ejemplo, el uso de un LLM (de esta lista) para una aplicación de diagnóstico médico a pacientes a través de chat sea un caso de uso, a priori, muy peligroso donde se desconoce por completo la capacidad del modelo para esta tarea. De la misma forma que usar un LLM para hacerle preguntas sobre burocracia española, o básicamente cualquier caso de uso que se le quiera dar.